In [1]:
# Importing Libraries
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import torch
from curses import meta
from locale import normalize
import os.path as osp
from typing import final
from xxlimited import new
import os, datetime
import torch
import time
import os, datetime
from torch_geometric.datasets import AMiner
from tqdm import tqdm
# import torch.multiprocessing
# torch.multiprocessing.set_sharing_strategy('file_system')
from model import MetaPath2Vec # modified Metapath2vec for the path embedding generation
import math

from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, precision_recall_curve

In [2]:
# embedding_dir = "/home/andrewngo/Desktop/MLTracker/model_20220217192920"
embedding_dir = "/home/andrewngo/Desktop/MLTracker/model_CUC_20220311120344" # model folder
graph_dir = '/home/andrewngo/Desktop/MLTracker/graph_data_20220305160351'
model = torch.load(embedding_dir + "/model.pt")
data = torch.load(graph_dir + '/graph_data_torch.pt')
metapath_strat = "CUC"
metapath = model.metapath
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
def generate_metapath_sampling(sample=20000):
    node_num = len(metapath) + 1
    model_sample = MetaPath2Vec(data.edge_index_dict, embedding_dim=128,
                        metapath=metapath, walk_length=node_num, context_size=node_num,
                        walks_per_node=1, num_negative_samples=1,
                        sparse=True).to(device)
    loader_sample = model_sample.loader(batch_size=1, shuffle=False, num_workers=1)
    optimizer = torch.optim.SparseAdam(list(model_sample.parameters()), lr=0.001)
    print(model_sample.start)
    print(model_sample.end)
    model_sample.train()
    temp = []
    for i, (pos_rw, neg_rw) in enumerate(tqdm(loader_sample)):
        temp.append(pos_rw.to(device)[0])
        if i > sample:
            break
    path = temp

    new_path = []
    print(path[1])
    for i in range(len(path)):
        if int(list(path[i])[-1]) != int(list(path[i])[-2]):
            new_path.append(path[i])
    # print(len(new_path))
    path = new_path


    temp = []
    for i in range(len(path)):


        if len(metapath)%2 == 0:
            temp.append(path[i][:math.ceil((len(metapath)+1)/2)])
            temp.append(path[i][math.ceil((len(metapath)+1)/2)-1:])
        else:
            print(path[i][:math.ceil((len(metapath)+1)/2)])
            temp.append(path[i][:math.ceil((len(metapath)+1)/2)])
            temp.append(path[i][math.ceil((len(metapath)+1)/2):])
    path = temp
    path = list(set(path))
    # print(path)
    print(len(path))
    return temp, model_sample.start, model_sample.end


In [4]:
len(metapath)

4

In [5]:
path, start_idx, end_idx = generate_metapath_sampling(100000) 

  0%|          | 0/15425 [00:00<?, ?it/s]

{'Computer': 0, 'User': 15425}
{'Computer': 15425, 'User': 27013}


100%|██████████| 15425/15425 [00:16<00:00, 961.23it/s] 


tensor([    1, 17020,   878, 19563,  2358], device='cuda:0')
30850


In [6]:
metapath_strat = "CUC"
def reindexing_path(model, malicious_path, path_type):
# ''' malicious_path of each node type are arranging from 0
# however, in metapath2vec implementation, they are convert to 
# a unique indexing (i.e, computer from 0 -> 32131, user from 32131 -> 42213). 
# This function will convert to metapath2vec reference indexing. '''
    new_path = []
    if path_type == "CUC":
        for i in malicious_path:
            first = i[0] + model.start["Computer"]
            sec = i[1] + model.start["User"]
            third = i[2] + model.start["Computer"]
            new_path.append((first, sec, third))
        # print(malicious_path)

    elif path_type == "UCAC":
        for i in malicious_path:
            first = i[0] + model.start["User"]
            sec = i[1] + model.start["Computer"]
            third = i[2] + model.start["Auth_Type"]
            forth = i[3] + model.start["Computer"]
            new_path.append((first, sec, third, forth))
    elif path_type == "UCCA":
        for i in malicious_path:
            first = i[0] + model.start["User"]
            sec = i[1] + model.start["Computer"]
            third = i[2] + model.start["Computer"]
            forth = i[3] + model.start["Auth_Type"]
            new_path.append((first, sec, third, forth))

    elif path_type == "UCC":
        for i in malicious_path:
            first = i[0] + model.start["User"]
            sec = i[1] + model.start["Computer"]
            third = i[2] + model.start["Computer"]
            new_path.append((first, sec, third))
    return new_path

dir_normal_path = "/normal_path_" + metapath_strat + ".pt"
normal_path = list(torch.load(graph_dir + dir_normal_path))
normal_path = reindexing_path(model, normal_path, metapath_strat)
normal_path_tensor = [torch.LongTensor(i).to(device) for i in normal_path]

# print(normal_path_tensor[1])
    
dir_all_malicious_path = "/all_malicious_path_" + metapath_strat + ".pt"
all_malicious_path = list(torch.load(graph_dir + dir_all_malicious_path))
all_malicious_path = reindexing_path(model, all_malicious_path, metapath_strat)
all_malicious_path_tensor = [torch.LongTensor(i).to(device) for i in all_malicious_path]

# print(all_malicious_path_tensor)

dir_train_val_malicious_path = "/train_val_malicious_path_" + metapath_strat + ".pt"
train_val_malicious_path = list(torch.load(graph_dir + dir_train_val_malicious_path))
train_val_malicious_path = reindexing_path(model, train_val_malicious_path, metapath_strat)
train_val_malicious_path_tensor = [torch.LongTensor(i).to(device) for i in train_val_malicious_path]


dir_test_malicious_path = "/test_malicious_path_" + metapath_strat + ".pt"
test_malicious_path = list(torch.load(graph_dir + dir_test_malicious_path))
test_malicious_path = reindexing_path(model, test_malicious_path, metapath_strat)
test_malicious_path_tensor = [torch.LongTensor(i).to(device) for i in test_malicious_path]

# print(malicious_path)
labels = [0 for i in range(len(path))] + [1 for i in range(len(all_malicious_path))]
path = path + all_malicious_path_tensor


out = []
out_index_path = dict()
for i in range(len(path)):
    out_index_path[len(out)] = path[i]
    out.append(model.get_embedding(path[i]))
# print(out[path[1]].size())
# print(out[1])


out_normal = []
out_normal_index_path = dict()
for i in range(len(normal_path)):
    out_normal_index_path[len(out_normal)] = normal_path[i]
    out_normal.append(model.get_embedding(normal_path_tensor[i]))


out_mal_train_val = []
out_mal_train_val_index_path = dict()
for i in range(len(train_val_malicious_path)):
    out_mal_train_val_index_path[len(out_mal_train_val)] = train_val_malicious_path[i]
    out_mal_train_val.append(model.get_embedding(train_val_malicious_path_tensor[i]))


out_mal_test = []
out_mal_test_index_path = dict()
for i in range(len(test_malicious_path)):
    out_mal_test_index_path[len(out_mal_test)] = test_malicious_path[i]
    out_mal_test.append(model.get_embedding(test_malicious_path_tensor[i]))


In [7]:
len(out_normal_index_path)

588958

In [8]:
store_directory = embedding_dir
torch.save(path, store_directory + '/path_1.pt')
torch.save(labels, store_directory + '/path_labels_1.pt')

torch.save(out, store_directory + '/out_1.pt')
torch.save(out_index_path, store_directory + '/out_index_path_1.pt')

torch.save(out_normal, store_directory + '/out_normal_1.pt')
torch.save(out_normal_index_path, store_directory + '/out_normal_index_path_1.pt')

torch.save(out_mal_train_val, store_directory + '/out_mal_train_val_1.pt')
torch.save(out_mal_train_val_index_path, store_directory + '/out_mal_train_val_index_path_1.pt')

torch.save(out_mal_test, store_directory + '/out_mal_test_1.pt')
torch.save(out_mal_test_index_path, store_directory + '/out_mal_test_index_path_1.pt')